In [12]:
import cv2
import numpy as np

def detect_gesture_background_subtraction(test_video_path, output_video_path):
  """
  This function detects gestures in a test video using background subtraction.

  Args:
      test_video_path (str): Path to the video containing random gestures.
      output_video_path (str): Path to save the annotated output video.
  """

  # Create Background Subtractor (MOG2 - Mixture of Gaussians)
  fgbg = cv2.createBackgroundSubtractorMOG2()

  cap = cv2.VideoCapture(test_video_path)
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

  kernel = np.ones((5, 5), np.uint8)  # Erosion/Dilation kernel

  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break

    # Apply background subtraction
    fgmask = fgbg.apply(frame)

    # Erode and Dilate to remove noise
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    fgmask = cv2.dilate(fgmask, kernel, iterations=3)

    # Find contours in the foreground mask
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours are detected
    if contours:
      # Assuming the largest contour corresponds to the hand/gesture
      largest_contour = max(contours, key=cv2.contourArea)

      # Draw bounding rectangle around the largest contour
      x, y, w, h = cv2.boundingRect(largest_contour)
      cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

      # Write "DETECTED" on the frame
      text_position = (x + 5, y + 20)
      cv2.putText(frame, 'DETECTED', text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Write the annotated frame to the output video
    out.write(frame)

  # Release resources
  cap.release()
  out.release()
  cv2.destroyAllWindows()

# Path configuration (Modify as needed)
test_video_path = '/content/vecteezy_slow-motion-asian-sportswoman-wearing-black-sportswear_8777747.mov'
output_video_path = 'output_video.avi'

# Call the detect_gesture function
detect_gesture_background_subtraction(test_video_path, output_video_path)





